In [ ]:
import pandas as pd
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory

# TODO: Make this nicer.
df = pd.DataFrame.from_records(
    {k: v.__dict__ for k, v in get_pretrained_saes_directory().items()}
).T
df.drop(
    columns=[
        "expected_var_explained",
        "expected_l0",
        "config_overrides",
        "conversion_func",
    ],
    inplace=True,
)
df.query("model == 'gemma-2-2b'")
# Each row is a "release" which has multiple SAEs which may have different configs / match different hook points in a model.

In [ ]:
from sae_lens import SAE, HookedSAETransformer

# sae_1,_,_ = SAE.from_pretrained("gemma-scope-2b-pt-mlp-canonical","layer_0/width_65k/canonical", device="mps")
sae_1, _, _ = SAE.from_pretrained(
    "gpt2-small-res-jb", "blocks.11.hook_resid_post", device="mps"
)
# model = HookedSAETransformer.from_pretrained(sae_1.cfg.model_name)

In [ ]:
from sae_lens import SAE, HookedSAETransformer
from sae_lens import SAE, HookedSAETransformer
import plotly.express as px
from sae_lens.analysis.neuronpedia_integration import (
    open_neuronpedia_feature_dashboard,
    get_neuronpedia_quick_list,
)


model = HookedSAETransformer.from_pretrained_no_processing(
    "gpt2-small",
    fold_ln=True,
    center_unembed=True,
    center_writing_weights=False,
)

sae_1, _, _ = SAE.from_pretrained(
    "gpt2-small-res-jb", "blocks.11.hook_resid_post", device="mps"
)

test_text = """ outscored the Pacers by just 16 points in the series. There has never been a smaller margin in a four-game sweep. — Dan Feldman (@DanFeldmanNBA) April 23, 2017

Why did it play out this way? Well, there were quite a few mitigating factors.

First, the Pacers played really, really well. Paul George played like a top-five player in this series, and the Cavaliers declined to put their best defensive weapon, LeBron James on him.

George averaged 28.0 points, 8.8 rebounds and 7.3 assists for the series, with his assists"""

text_df = pd.DataFrame(enumerate(model.to_str_tokens(test_text)))
text_df.columns = ["token_id", "token"]
text_df["token_id"] = text_df["token_id"].astype(int)
text_df["token"] = text_df["token"].astype(str)
# find the token "Dan"
display(text_df.query("token == 'Dan'"))

_, cache = model.run_with_cache_with_saes(test_text, saes=[sae_1])
px.line(
    cache[sae_1.cfg.hook_name + ".hook_sae_acts_post"][0, 32, :].cpu().numpy()
).show()

In [ ]:
from sae_lens import SAE, HookedSAETransformer

import plotly.express as px
from sae_lens.analysis.neuronpedia_integration import (
    open_neuronpedia_feature_dashboard,
    get_neuronpedia_quick_list,
)


model = HookedSAETransformer.from_pretrained_no_processing(
    "gpt2-small",
    fold_ln=True,
    center_unembed=True,
    center_writing_weights=True,
)

sae_1, _, _ = SAE.from_pretrained(
    "gpt2-small-res-jb", "blocks.11.hook_resid_post", device="mps"
)

test_text = """ outscored the Pacers by just 16 points in the series. There has never been a smaller margin in a four-game sweep. — Dan Feldman (@DanFeldmanNBA) April 23, 2017

Why did it play out this way? Well, there were quite a few mitigating factors.

First, the Pacers played really, really well. Paul George played like a top-five player in this series, and the Cavaliers declined to put their best defensive weapon, LeBron James on him.

George averaged 28.0 points, 8.8 rebounds and 7.3 assists for the series, with his assists"""

text_df = pd.DataFrame(enumerate(model.to_str_tokens(test_text)))
text_df.columns = ["token_id", "token"]
text_df["token_id"] = text_df["token_id"].astype(int)
text_df["token"] = text_df["token"].astype(str)
# find the token "Dan"
display(text_df.query("token == 'Dan'"))

_, cache = model.run_with_cache_with_saes(test_text, saes=[sae_1])
px.line(
    cache[sae_1.cfg.hook_name + ".hook_sae_acts_post"][0, 32, :].cpu().numpy()
).show()

## Now try Gemma

In [50]:
for i in range(10):
    sae_1, _, _ = SAE.from_pretrained(
        "gemma-scope-2b-pt-mlp-canonical",
        f"layer_{i}/width_65k/canonical",
        device="mps",
    )

In [ ]:
from sae_lens import SAE, HookedSAETransformer
import plotly.express as px

model = HookedSAETransformer.from_pretrained_no_processing(
    "gemma-2-2b",
    fold_ln=True,
    center_unembed=True,
    center_writing_weights=False,
)

# sae_1,_,_ = SAE.from_pretrained("gpt2-small-res-jb","blocks.11.hook_resid_post", device="mps")

test_text = """ damn"""

# text_df = pd.DataFrame(enumerate(model.to_str_tokens(test_text)))
# text_df.columns = ["token_id", "token"]
# text_df["token_id"] = text_df["token_id"].astype(int)
# text_df["token"] = text_df["token"].astype(str)
# # find the token "Dan"
# display(text_df.query("token == 'Dan'"))

_, cache = model.run_with_cache_with_saes(test_text, saes=[sae_1])
px.line(
    cache[sae_1.cfg.hook_name + ".hook_sae_acts_post"][0, -1, :].cpu().numpy()
).show()

In [ ]:
from sae_lens import SAE, HookedSAETransformer
import plotly.express as px

model = HookedSAETransformer.from_pretrained_no_processing(
    "gemma-2-2b",
    fold_ln=True,
    center_unembed=True,
    center_writing_weights=True,
)

test_text = """ damn"""

_, cache = model.run_with_cache_with_saes(test_text, saes=[sae_1])
px.line(
    cache[sae_1.cfg.hook_name + ".hook_sae_acts_post"][0, -1, :].cpu().numpy()
).show()

## Hit Server

In [ ]:
sae_1.cfg.neuronpedia_id.split("/")[1]

In [ ]:
sae_1.cfg.model_name

In [ ]:
sae_1.cfg.model_name

In [ ]:
import requests

# Test activations_all endpoint
base_url = "http://127.0.0.1:5000"
url = f"{base_url}/activations-all"

payload = {
    "text": test_text,
    "model": "gpt2-small",
    "source_set": "res-jb",
    "selected_layers": ["12-res-jb"],
    "secret": "secret",
    "sort_indexes": [32],
}

response = requests.post(url, json=payload)
response.json()

In [ ]:
import requests

# Test activations_all endpoint
base_url = "http://127.0.0.1:5000"
url = f"{base_url}/activations-all"

payload = {
    "text": " damn",
    "model": sae_1.cfg.model_name + "-it",
    "source_set": "gemmascope-mlp-65k",
    "selected_layers": ["0-gemmascope-mlp-65k"],
    "secret": "secret",
    "sort_indexes": [],
}

response = requests.post(url, json=payload)
response.json()